In [3]:
import os
from groq import Groq
from elasticsearch import Elasticsearch
import requests 
from tqdm.auto import tqdm


client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

In [4]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [12]:
def build_prompt(query, search_results):
    context_template = """
    Q: {question}
    A: {text}
        """.strip()
    context = ''
    
    for hit in search_results['hits']['hits']:
        context += context_template.format(question = hit['_source']['question'], text = hit['_source']['text']) + '\n' + '\n'
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
        
    QUESTION: {question}
        
    CONTEXT:
    {context}
    """.strip()
    prompt = prompt_template.format(question=query, context=context)
    print(len(prompt))
    return prompt

In [23]:
def llm(prompt):
    response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="gemma2-9b-it",
    )
    return response.choices[0].message.content.strip()

In [33]:
es_client = Elasticsearch('http://localhost:9200')
# index_name = "course-questions"
# es_client.indices.create(index=index_name, body=index_settings)
# for doc in tqdm(documents):
#     es_client.index(index=index_name, document=doc)

In [34]:
def search(query):
    index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
                "course": {"type": "keyword"} 
            }
        }
    }
    search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text", "section"],
                    "type": "best_fields"
                    }
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)
    return response

In [38]:
def rag(query):
    search_results = search(query)
    print(search_results)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [41]:
query = "I just dicovered the course, can i join now?"

In [42]:
rag(query)

{'took': 11, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 3500, 'relation': 'eq'}, 'max_score': 80.367874, 'hits': [{'_index': 'course-questions', '_id': 'nhfjepcB6jH3fv-e0J8L', '_score': 80.367874, '_source': {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'General course-related questions', 'question': 'Course - Can I still join the course after the start date?', 'course': 'data-engineering-zoomcamp'}}, {'_index': 'course-questions', '_id': 'Uhf3epcB6jH3fv-eQKNm', '_score': 80.367874, '_source': {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'Gene

'Yes, you can still join the course.'